In [14]:
%pip install jupyter pandas scikit-learn matplotlib seaborn
%pip install nltk
%pip install umap-learn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#  Step 1: Load and Explore the Data



In [3]:
import pandas as pd

# Load product data (assuming it's a JSON file exported from MongoDB)
df = pd.read_json("../JSON/products.json")

# Preview the data
df.head()

,category,title,price,description
0,top,Silk Camisole,46.50,Featuring a modern cut and wool blend fabric t...
1,top,Crop Top,85.88,Soft and breathable modal construction with st...
2,top,Silk Camisole,107.06,Crafted from high-quality silk that's built to...
3,top,Patterned Blouse,136.74,Featuring a modern cut and polyester fabric th...
4,top,Tunic,33.05,Crafted from high-quality modal that's built t...


#  Step 2: Preprocess Data

In [6]:
df_encoded = pd.get_dummies(df, columns=["category"])
print(df_encoded.head())

              title   price  \
0     Silk Camisole   46.50   
1          Crop Top   85.88   
2     Silk Camisole  107.06   
3  Patterned Blouse  136.74   
4             Tunic   33.05   

                                         description  category_accessory  \
0  Featuring a modern cut and wool blend fabric t...               False   
1  Soft and breathable modal construction with st...               False   
2  Crafted from high-quality silk that's built to...               False   
3  Featuring a modern cut and polyester fabric th...               False   
4  Crafted from high-quality modal that's built t...               False   

   category_bottom  category_shoes  category_top  
0            False           False          True  
1            False           False          True  
2            False           False          True  
3            False           False          True  
4            False           False          True  


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_encoded["price"] = scaler.fit_transform(df_encoded[["price"]])

In [8]:
feature_columns = [col for col in df_encoded.columns if col not in ["title", "description", "_id"]]
X = df_encoded[feature_columns]


In [9]:
X.head()


,price,category_accessory,category_bottom,category_shoes,category_top
0,0.191659,False,False,False,True
1,0.399293,False,False,False,True
2,0.510967,False,False,False,True
3,0.667458,False,False,False,True
4,0.120742,False,False,False,True


#  Step 3: Fit the Nearest Neighbors Model

In [10]:
from sklearn.neighbors import NearestNeighbors

# Use cosine similarity for better results with mixed features
nn = NearestNeighbors(n_neighbors=6, metric='cosine')  # 6 because the first result will be the item itself
nn.fit(X)


,n_neighbors,6
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [18]:
# Helper function to get similar items
def get_similar_items(item_index, X, df, model, n=5):
    distances, indices = model.kneighbors([X.iloc[item_index]], n_neighbors=n+1)
    print("Cosine Distances:", distances[0][1:])
    # Skip the first one (it's the item itself)
    similar_indices = indices[0][1:]
    similar_items = df.iloc[similar_indices]
    
    return similar_items


In [19]:
# Test for random item
# Pick an item index (e.g. 0 for the first product)
item_index = 0
print("Target item:", df.iloc[item_index]["title"])

similar_items = get_similar_items(item_index, X, df, nn)
similar_items[["title", "category", "price"]]


Target item: Silk Camisole
Cosine Distances: [3.73874754e-07 5.81296696e-06 4.39722332e-04 4.62907321e-04
 5.75939601e-04]


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,title,category,price
74,Tank Top,top,46.33
22,Patterned Blouse,top,45.83
55,Silk Camisole,top,40.70
42,Long Sleeve Thermal,top,52.52
12,Silk Camisole,top,53.22


# Add UMAP Coordinates

In [15]:
import umap.umap_ as umap

# Initialize and fit UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine')
embedding = reducer.fit_transform(X)

# Add UMAP coordinates to the DataFrame
df['umap_x'] = embedding[:, 0]
df['umap_y'] = embedding[:, 1]

# Preview
df[['title', 'category', 'price', 'umap_x', 'umap_y']].head()


,title,category,price,umap_x,umap_y
0,Silk Camisole,top,46.50,17.498072,-6.178461
1,Crop Top,top,85.88,11.857539,-4.641145
2,Silk Camisole,top,107.06,9.647942,-4.241848
3,Patterned Blouse,top,136.74,7.247543,-3.849845
4,Tunic,top,33.05,16.929638,-7.323118


In [16]:
df_export = df[['title', 'category', 'price', 'umap_x', 'umap_y']]
df_export.to_json('../JSON/umap_products.json', orient='records', indent=2)


# Export Similar Items table for lookup

In [17]:
import json

similar_lookup = {}
for i in range(len(df)):
    similar = get_similar_items(i, X, df, nn)
    similar_lookup[df.iloc[i]["title"]] = similar["title"].tolist()

with open("../JSON/similar_items.json", "w") as f:
    json.dump(similar_lookup, f, indent=2)


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali